In [1]:
# AutoEncoders

# Importing the libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [53]:
# # Importing the dataset
data = pd.read_csv('all_sem_grades.csv')
data= data.drop('NO.',axis=1)
data.columns


Index(['MA101', 'PH100', ' BE10105', 'BE110', ' BE103', ' EC100', ' PH110',
       ' EC110', ' CS110', 'CY100', 'BE100', 'EE100', 'CY110', 'EE110',
       'MA102', 'BE102', 'CS100', 'CS120', 'CS202', 'IT202', 'IT204', 'HS200',
       'MA202', 'CS208', 'IT232', 'IT234', 'CS305', 'IT301', 'IT303', 'IT305',
       'IT307', 'IT341', 'IT331', 'IT333', 'IT367', 'IT363', 'HS300', 'CS304',
       'IT304', 'IT306', 'IT364', 'IT302', 'IT332', 'IT334', 'IT352', 'IT366'],
      dtype='object')

In [54]:
# Preparing the training set and the test set
training_set = pd.read_csv('all_train.csv')
training_set= training_set.drop('NO.',axis=1)
training_set = np.array(training_set, dtype = 'int')#user_id,movie_id,ratings
test_set = pd.read_csv('all_test.csv')
test_set= test_set.drop('NO.',axis=1)
test_set = np.array(test_set, dtype = 'int')
test_set.shape
#training_set.shape

(10, 46)

In [55]:
test_set.shape
#training_set.shape

(10, 46)

In [56]:
#nb_students = int(max(max(training_set[:,0]), max(test_set[:,0])))
#nb_courses = int(max(max(training_set[:,1]), max(test_set[:,1])))
nb_students = max(training_set.shape[0], test_set.shape[0])-1
nb_courses = max(training_set.shape[1], test_set.shape[1])
nb_students
#nb_courses

36

In [57]:

# Converting the data into Torch tensors
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)


In [58]:
# Creating the architecture of the Neural Network
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_courses, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_courses)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x.view(x.size(0), -1) 
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)


In [59]:
# Training the SAE
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_students):
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data > 0) > 0:
            output = sae(input)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_courses/float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            
            train_loss += np.sqrt(loss.item()*mean_corrector)

            s += 1.
            optimizer.step()
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))


epoch: 1 loss: 2.0663810213106097e+18
epoch: 2 loss: 2.0663810213106097e+18
epoch: 3 loss: 2.0663810213106097e+18
epoch: 4 loss: 2.0663810213106097e+18
epoch: 5 loss: 2.0663810213106097e+18
epoch: 6 loss: 2.0663810213106097e+18
epoch: 7 loss: 2.0663810213106097e+18
epoch: 8 loss: 2.0663810213106097e+18
epoch: 9 loss: 2.0663810213106097e+18
epoch: 10 loss: 2.0663810213106097e+18
epoch: 11 loss: 2.0663810213106097e+18
epoch: 12 loss: 2.0663810213106097e+18
epoch: 13 loss: 2.0663810213106097e+18
epoch: 14 loss: 2.0663810213106097e+18
epoch: 15 loss: 2.0663810213106097e+18
epoch: 16 loss: 2.0663810213106097e+18
epoch: 17 loss: 2.0663810213106097e+18
epoch: 18 loss: 2.0663810213106097e+18
epoch: 19 loss: 2.0663810213106097e+18
epoch: 20 loss: 2.0663810213106097e+18
epoch: 21 loss: 2.0663810213106097e+18
epoch: 22 loss: 2.0663810213106097e+18
epoch: 23 loss: 2.0663810213106097e+18
epoch: 24 loss: 2.0663810213106097e+18
epoch: 25 loss: 2.0663810213106097e+18
epoch: 26 loss: 2.0663810213106097

In [60]:
# Testing the SAE
test_loss = 0
s = 0.
for id_user in range(nb_students):
    input = Variable(training_set[id_user]).unsqueeze(0)
    #target = Variable(test_set[id_user]).unsqueeze(0)
for id_user in range(test_set.shape[0]):
    target = Variable(test_set[id_user]).unsqueeze(0)


    if torch.sum(target.data > 0) > 0:
        output = sae(input)
        target.require_grad = False
        output[target == 0] = 0
        loss = criterion(output, target)
        mean_corrector = nb_courses/float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.item()*mean_corrector)

        s += 1.
print('test loss: '+str(test_loss/s))

test loss: 2.0880597246636442e+18


In [15]:
columns=data.columns
c=columns.tolist()
d=pd.DataFrame(c)
d

,0
0,MA101
1,PH100
2,BE10105
3,BE110
4,BE103
5,EC100
6,PH110
7,EC110
8,CS110
9,CY100


In [61]:
student_id = 2

student_grade = training_set.data.numpy()[student_id, :].reshape(-1,1)
user_target = test_set.data.numpy()[student_id, :].reshape(-1,1)
 
user_input = Variable(training_set[student_id]).unsqueeze(0)
predicted = sae(user_input)
predicted = predicted.data.numpy().reshape(-1,1)
# Join all info in one dataset
result_array = np.hstack([d, user_target, predicted])
result_array = result_array[result_array[:, 1] > 0]
result_df = pd.DataFrame(data=result_array, columns=['courses', 'Target Rating', 'Predicted'])

In [63]:
result_df.head(40)

,courses,Target Rating,Predicted
0,MA101,7,5.50504
1,PH100,6,7.16058
2,BE10105,5,7.5648
3,BE110,9,8.43282
4,BE103,7,6.11273
5,EC100,6,6.5294
6,PH110,10,9.51373
7,EC110,9,11.0226
8,CS110,9,10.1607
9,CY100,6,6.09016
